## Aprendizado Profundo - UFMG

## Problemas

Como vimos acima, há muitos passos na criação e definição de uma nova rede neural.
A grande parte desses ajustes dependem diretamente do problemas.

Abaixo, listamos alguns problemas. Todos os problemas e datasets usados vem do [Center for Machine Learning and Intelligent Systems](http://archive.ics.uci.edu/ml/datasets.php).


**Seu objetivo é determinar e implementar um modelo para cada problema.**

Isso inclui definir uma arquitetura (por enquanto usando somente camadas [Densas](https://mxnet.incubator.apache.org/api/python/gluon/nn.html#mxnet.gluon.nn.Dense), porém podemos variar as ativações -- [Sigmoid](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.Symbol.sigmoid), [Tanh](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.Symbol.tanh), [ReLU](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.Symbol.relu), [LeakyReLU, ELU, SeLU, PReLU, RReLU](https://mxnet.incubator.apache.org/api/python/symbol/symbol.html#mxnet.symbol.LeakyReLU)), uma função de custo ( [L1](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.L2Loss), [L2](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.L1Loss),[ Huber](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.HuberLoss), [*Cross-Entropy*](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.SoftmaxCrossEntropyLoss), [Hinge](https://mxnet.incubator.apache.org/api/python/gluon/loss.html#mxnet.gluon.loss.HingeLoss)), e um algoritmo de otimização ([SGD](https://mxnet.incubator.apache.org/api/python/optimization/optimization.html#mxnet.optimizer.SGD), [Momentum](https://mxnet.incubator.apache.org/api/python/optimization/optimization.html#mxnet.optimizer.SGD), [RMSProp](https://mxnet.incubator.apache.org/api/python/optimization/optimization.html#mxnet.optimizer.RMSProp), [Adam](https://mxnet.incubator.apache.org/api/python/optimization/optimization.html#mxnet.optimizer.Adam)).

A leitura do dado assim como a função de treinamento já estão implementados.

Esse pequeno bloco de código abaixo é usado somente para instalar o MXNet para CUDA 10. Execute esse bloco somente uma vez e ignore possíveis erros levantados durante a instalação.

**ATENÇÃO: a alteração deste bloco pode implicar em problemas na execução dos blocos restantes!**

In [1]:
! pip install mxnet-cu100

     |████████████████████████████████| 540.1MB 35kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


# Preâmbulo

In [0]:
# imports basicos

from mxnet import autograd
from mxnet import gluon
from mxnet import init
from mxnet import nd

from mxnet.gluon import data as gdata
from mxnet.gluon import loss as gloss
from mxnet.gluon import nn
from mxnet.gluon import utils as gutils

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import mxnet as mx
import numpy as np

import os
import sys
import time

In [0]:
import matplotlib.pyplot as plt
plt.ion()

In [4]:
# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
ctx

gpu(0)

In [0]:
# funções básicas

def load_array(features, labels, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset = gluon.data.ArrayDataset(features, labels)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)

def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
    
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs, type='regression'):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        if type == 'regression':
            print('epoch %d, train loss %.4f, test loss %.4f, time %.1f sec'
                    % (epoch + 1, train_l_sum / n, test_loss, time.time() - start))
        else:
            print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, ' \
                  'test acc %.3f, time %.1f sec' % \
                  (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, test_acc, time.time() - start))
          
        
# funcao usada para teste
def test(net, test_iter):
    print('testing on', ctx)
    first = True
    for X in test_iter:
        X = X.as_in_context(ctx)
        y_hat = net(X)
        if first is True:
            pred_logits = y_hat
            pred_labels = y_hat.argmax(axis=1)
            first = False
        else:
            pred_logits = nd.concat(pred_logits, y_hat, dim=0)
            pred_labels = nd.concat(pred_labels, y_hat.argmax(axis=1), dim=0)

    return pred_logits.asnumpy(), pred_labels.asnumpy()

## Problema 1

Neste problema, você receberá 7 *features* extraídas de poços de petróleo ('BRCALI', 'BRDENS', 'BRDTP', 'BRGR', 'BRNEUT', 'BRRESC', 'BRRESP') e deve predizer o tipo de rocha.

### Treino e Validação

Primeiro, vamos modelar uma rede neural e treiná-la.
Usamos o dado de treino carregado no próximo bloco para convergir o modelo e o dado de validação para avaliar quão bom ele estão. 

In [6]:
# download do dataset
!wget https://www.dropbox.com/s/ujnqxh6l43tlbdi/poco_1.prn
X = np.loadtxt('poco_1.prn', skiprows=11, usecols=(1,2,3,4,5,6,7), dtype=np.float32)
y = np.loadtxt('poco_1.prn', skiprows=11, usecols=8, dtype=np.str)
print(y)
print(set(y))
le = preprocessing.LabelEncoder()
le.fit(list(set(y)))
y_t = le.transform(y)

print(X[0, :])
print(y[0], y_t[0])
print(y[960], y_t[960])
train_features, test_features, train_labels, test_labels = train_test_split(X, y_t, test_size=0.33)

def load_array(features, labels, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset = gluon.data.ArrayDataset(features, labels)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
  
batch_size = 10
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

--2019-09-12 17:26:28--  https://www.dropbox.com/s/ujnqxh6l43tlbdi/poco_1.prn
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ujnqxh6l43tlbdi/poco_1.prn [following]
--2019-09-12 17:26:29--  https://www.dropbox.com/s/raw/ujnqxh6l43tlbdi/poco_1.prn
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc0414dd52f6561961141a4e9028.dl.dropboxusercontent.com/cd/0/inline/AoZq9yVFkbZXMnx4-m2sELhAztYEH0JPRk2Czk4I-n9QtdKL0wzRaqr9S0iQhYWaioVgsSV7mOC3TaEJilzYarPaVoLdtVL0NhUFqA9CtOlkVA/file# [following]
--2019-09-12 17:26:30--  https://uc0414dd52f6561961141a4e9028.dl.dropboxusercontent.com/cd/0/inline/AoZq9yVFkbZXMnx4-m2sELhAztYEH0JPRk2Czk4I-n9QtdKL0wzRaqr9S0iQhYWaioVgsSV7mOC3TaEJilzYarPaVoLdtVL0NhUFqA9CtOlkVA/file
Resolving uc0414dd52f6

In [7]:
net = nn.Sequential()
net.add(nn.Dense(64, activation='relu'),
        nn.Dropout(0.2),
        nn.Dense(32, activation='relu'),
        nn.Dense(16, activation='relu'),
        nn.Dense(2))

weight_decay = 0.01

net.initialize(init.Normal(sigma=0.01), ctx=ctx)
num_epochs = 100
l2_norm = gloss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate' : 0.0001, 'wd' : weight_decay})
train_validate(net, train_iter, test_iter, batch_size, trainer, l2_norm, ctx, num_epochs, type='classification')

training on gpu(0)
epoch 1, train loss 0.6917, train acc 0.650, test loss 0.6902, test acc 0.665, time 0.9 sec
epoch 2, train loss 0.6872, train acc 0.666, test loss 0.6806, test acc 0.665, time 0.8 sec
epoch 3, train loss 0.6648, train acc 0.666, test loss 0.6472, test acc 0.665, time 0.8 sec
epoch 4, train loss 0.6124, train acc 0.666, test loss 0.5574, test acc 0.665, time 0.8 sec
epoch 5, train loss 0.4943, train acc 0.666, test loss 0.4697, test acc 0.665, time 0.8 sec
epoch 6, train loss 0.4399, train acc 0.690, test loss 0.4330, test acc 0.848, time 0.8 sec
epoch 7, train loss 0.4071, train acc 0.877, test loss 0.4042, test acc 0.873, time 0.8 sec
epoch 8, train loss 0.3744, train acc 0.889, test loss 0.3661, test acc 0.891, time 0.8 sec
epoch 9, train loss 0.3384, train acc 0.899, test loss 0.3274, test acc 0.905, time 0.8 sec
epoch 10, train loss 0.3063, train acc 0.905, test loss 0.3063, test acc 0.900, time 0.8 sec
epoch 11, train loss 0.2813, train acc 0.905, test loss 0.30

## Problema 2

Neste problema, você receberá várias *features* (como altura média, inclinação, etc) descrevendo uma região e o modelo deve predizer qual o tipo da região (floresta, montanha, etc).

In [0]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
!gzip covtype.data.gz
data = np.genfromtxt('covtype.data', delimiter=',', dtype=np.float32)

print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0]k)
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)

def load_array(features, labels, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset = gluon.data.ArrayDataset(features, labels)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
  
batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

## Problema 3

Neste problema, você receberá 90 *features* extraídas de diversas músicas (datadas de 1922 até 2011) e deve predizer o ano de cada música.

In [0]:
# download do dataset
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
!unzip YearPredictionMSD.txt.zip
data = np.genfromtxt('YearPredictionMSD.txt', delimiter=',', dtype=np.float32)

print(data[0, :])
X, y = data[:, 1:], data[:, 0]
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)

def load_array(features, labels, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset = gluon.data.ArrayDataset(features, labels)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
  
batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)